<a href="https://colab.research.google.com/github/stevenrdz/IR-SOES/blob/master/WMD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from time import time
start_nb = time()

import pandas as pd
from nltk.corpus import stopwords
import re
from nltk import word_tokenize
import string

es_stops = set(stopwords.words('spanish'))

In [ ]:
start = time()
dataset = pd.read_csv('//content/gdrive/My Drive/df.csv',
                      sep=';', encoding='utf-8-sig')
dataset_size = len(dataset)


print('Dataset generado. %.2f segundos' % (time() - start))
dataset.head()

Dataset generado. 0.89 segundos


,Id,Title,Body
0,286498,Debugear código Django utilizando Docker,<p>Les cuento.\r\nEstoy realizando un proyecto...
1,226916,En el file.conf muestra Error usando Superviso...,<p>Al configurar el file.conf<br>\r\nruta : su...
2,161561,Ejecutar navegador en pantalla completa al ini...,<p>Estoy realizando un <em>proyecto Domotico</...
3,230560,Pagina web en blanco,<p>Buen dia \r\nNo se por que mi pagina web sa...
4,309146,Obtengo el error I/Choreographer: Skipped 110 ...,<p>Estoy trabajando en android studio intentan...


In [ ]:
def preprocess(doc):
    code = re.compile('[¿?""(),:'']') # Eliminación de signos de puntuacion.
    doc = re.sub(code, '', doc)
    punt = re.compile('[./@_]') # Eliminación de signos de puntuacion.
    doc = re.sub(punt, ' ', doc)
    code = re.compile('<pre><code>[\s\S]*</code></pre>') # Eliminación de codigo.
    doc = re.sub(code, '', doc)
    tag = re.compile('<.*?>') # Eliminación de etiquetas.
    doc = re.sub(tag, '', doc)
    doc = doc.lower()  # Minuscula todo el texto.
    doc = word_tokenize(doc)  # Dividir en palabras.
    doc = [w for w in doc if not w in es_stops]  # Eliminar stopwords.
    # doc = [w for w in doc if w.isalpha()]  # Eliminar numbers and punctuation.
    return doc

In [ ]:
start = time()
dataset['Processed_Title'] = dataset.apply(
        lambda dataset: preprocess(dataset['Title']), axis=1)
dataset['Processed_Body'] = dataset.apply(
        lambda dataset: preprocess(dataset['Body']), axis=1)
print('Preprocesado completo. %.2f segundos' % (time() - start))
dataset.iloc[:,[3,4]]

Preprocesado completo. 1.14 segundos


,Processed_Title,Processed_Body
0,"[debugear, código, django, utilizando, docker]","[cuento, realizando, proyecto, django, utilizo..."
1,"[file, conf, muestra, error, usando, superviso...","[configurar, file, conf, ruta, sudo, nano, etc..."
2,"[ejecutar, navegador, pantalla, completa, inic...","[realizando, proyecto, domotico, siguientes, c..."
3,"[pagina, web, blanco]","[buen, dia, pagina, web, sale, blanco, alguien..."
4,"[obtengo, error, i, choreographer, skipped, 11...","[trabajando, android, studio, intentando, hace..."
...,...,...
1079,"[conexion, remota, sql, server, linux, mint, 19]","[instalado, sql, server, linux, mint, 19, base..."
1080,"[crear, base, datos, mssql-server, centos, 7, ...","[utilizando, mssql-server, centos, seguí, inst..."
1081,"[hacer, respaldo, base, datos, terminal]","[base, datos, ms, sql, server, 2017, express, ..."
1082,"[aplicación, windows, linux, wine]","[aplicación, desarrollada, vb, net, deve, xpre..."


In [ ]:
print(dataset['Processed_Body'][0])
print(dataset['Processed_Body'][5])

['cuento', 'realizando', 'proyecto', 'django', 'utilizo', 'pycharm', 'windows', '8', '1', 'levantar', 'servidor', 'web', 'base', 'datos', 'utilizo', 'docker', 'instalado', 'debian', 'linux', 'traves', 'maquina', 'virtual', 'instale', 'docker', 'windows', 'problemas', 'compatibilidad', 'problema', 'trabajar', 'manera', 'encuentro', 'forma', 'poder', 'debugear', 'código', 'poner', 'puntos', 'interrupcion', 'demás', 'buscando', 'lados', 'encontre', 'ninguna', 'solución', 'tal', 'vez', 'alguno', 'pudo', 'resolver', 'problema', 'dudas', 'dejo', 'archivo', 'docker-compose', 'yml', 'version', "'3", "4'", 'services', 'db', 'image', 'postgres', 'container', 'name', 'csuperior-postgres', 'environment', 'postgres', 'user', 'xxxxxxx', 'postgres', 'password', 'xxxxxxx', 'broker', 'image', 'rabbitmq', 'container', 'name', 'csuperior-broker', 'environment', '-', 'rabbitmq', 'default', 'user=xxxxxxx', '-', 'rabbitmq', 'default', 'pass=xxxxxxx', 'web', 'container', 'name', 'csuperior-web', 'volumes', '

In [ ]:
dataset['Full_Processed'] = dataset['Processed_Title'] + dataset['Processed_Body']

In [ ]:
from sklearn.model_selection import train_test_split 

In [ ]:
X = dataset['Processed_Title'] #Full_Processed Processed_Title

In [ ]:
train_p, test_p = train_test_split(X, test_size = 0.20, shuffle = False)

print("Ejemplos usados para entrenar: ", len(train_p))
print("Ejemplos usados para test: ", len(test_p))

Ejemplos usados para entrenar:  867
Ejemplos usados para test:  217


In [ ]:
processed_title = dataset['Processed_Title'].to_list() #Full_Processed Processed_Title Processed_Body
title = dataset['Title'].to_list()

In [ ]:
from gensim.models import Word2Vec
# Train Word2Vec on all post.
model_p = Word2Vec(processed_title, workers=3, size=100)

# Inicialización de WmdSimilarity.
from gensim.similarities import WmdSimilarity
num_best = 15
instancia_p = WmdSimilarity(train_p, model_p, num_best)
print(instancia_p)

WmdSimilarity<867 docs, 100 features>


In [ ]:
start = time()

msj_p = '¿Por qué no existe oficialmente un Docker para Sistemas operativos de 32 bits (x86)?'
bus_p = preprocess(msj_p)
print(bus_p)
sim_p = instancia_p[bus_p]
# print(len(sim_p)) 5
print ('Tiempo de ejecución: %.2f segundos en ejecutarse.' %(time() - start))

['existe', 'oficialmente', 'docker', 'sistemas', 'operativos', '32', 'bits', 'x86']
Tiempo de ejecución: 0.21 segundos en ejecutarse.


In [ ]:
print ('Búsqueda: ', msj_p)
print("")
for i in range(num_best): # (num_best)
    print (title[sim_p[i][0]],'---> sim = %.4f' % sim_p[i][1])

Búsqueda:  ¿Por qué no existe oficialmente un Docker para Sistemas operativos de 32 bits (x86)?

Por qué no existe oficialmente un Docker para Sistemas operativos de 32 bits (x86)? ---> sim = 1.0000
Designar directorio de docker de Informix ---> sim = 0.6193
¿Como se cambia la ip a la que se enlaza un contenedor docker? ---> sim = 0.5981
Docker container error Waiting for headers ---> sim = 0.5520
¿Existe alguna combinación de teclas para imprimir caracteres ASCII en Linux? ---> sim = 0.5453
Error con Plugin de nginx para Docker Compose ---> sim = 0.5437
Como comprobar si existe una palabra dentro de un fichero ---> sim = 0.5141
Debugear código Django utilizando Docker ---> sim = 0.5129
Mejor opción: Windows Docker vs Compilar en Linux? ---> sim = 0.5107
¿Porque si añado la ubicación de mi programa al PATH y luego lo ejecuto me dice que no existe en Ubuntu? ---> sim = 0.5016
¿Existe una manera estándar de limpiar la pantalla en C/C++? ---> sim = 0.5000
/boot/grub no existe pero /grub s

In [ ]:
# Error al instalar MySQL 8 en Ubuntu 16.04 ---> sim = 1.0000
# Error al instalar php 7,2 en ubuntu 16.04 ---> sim = 0.7468
# Ubuntu 16.02 lts error al instalar e desinstalar paquetes ---> sim = 0.7345
# ¿Por que cuando intento instalar el ssh en ubuntu me muestra error? ---> sim = 0.6593
# ¿Cómo instalar php-mbregex en ubuntu 16.04? ---> sim = 0.6302
# Error instalar xampp en linux ---> sim = 0.6237
# Error con "main" no definida en linux ubuntu ---> sim = 0.6172
# ¿Como puedo instalar PHP en Ubuntu? ---> sim = 0.6146
# Mysql error 1046: No database selected ---> sim = 0.6116
# Problema con el grub al instalar ubuntu ---> sim = 0.6080
# ¿Ayuda me da este error al hacer en consulta en Mysql Uncaught Error: Call to undefined function mysql_error()? ---> sim = 0.6062
# Error found when loading /home/hernan/.profile Ubuntu 18 ---> sim = 0.6062
# Google compute Engine, error al conectar remotamente al MySQL ---> sim = 0.6057
# Error al instalar en Red Hat ---> sim = 0.6054
# Problemas al instalar LAMP en ubuntu ---> sim = 0.6053

In [ ]:
search_results = []
for i in range(num_best): # (num_best)
  # print (title[sim_p[i][0]],'---> sim = %.4f' % sim_p[i][1])
  temp = {
            'title': str(title[sim_p[i][0]]),
            'similarity_score': str('%.4f' % sim_p[i][1]),
        }
  search_results.append(temp)

print(search_results[:])

[{'title': 'Error al instalar MySQL 8 en Ubuntu 16.04', 'similarity_score': '1.0000'}, {'title': 'Error al instalar php 7,2 en ubuntu 16.04', 'similarity_score': '0.7486'}, {'title': '¿Cómo instalar php-mbregex en ubuntu 16.04?', 'similarity_score': '0.6855'}, {'title': 'Ubuntu 16.02 lts error al instalar e desinstalar paquetes', 'similarity_score': '0.6616'}, {'title': 'CronJob Ubuntu 16.04/php7.0', 'similarity_score': '0.6125'}, {'title': 'VM Ubuntu 16.04 , No se inicia ningún Navegador web', 'similarity_score': '0.6107'}, {'title': '¿Por que cuando intento instalar el ssh en ubuntu me muestra error?', 'similarity_score': '0.6072'}, {'title': '¿Que falta por configurar? IP estática en Ubuntu 16.04', 'similarity_score': '0.6016'}, {'title': 'Error de OpenSSL usando composer en ubuntu 18.04', 'similarity_score': '0.5947'}, {'title': 'Error con "main" no definida en linux ubuntu', 'similarity_score': '0.5766'}, {'title': '¿Como puedo instalar PHP en Ubuntu?', 'similarity_score': '0.5686

In [ ]:
model_p.save('wmd.bin')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# Import saved Wordvec Embeddings
import gensim
w2v_model = gensim.models.word2vec.Word2Vec.load('/content/wmd.bin')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
w2v_model

In [ ]:
# Inicialización de WmdSimilarity.
from gensim.similarities import WmdSimilarity
num_best = 5
instancia_p = WmdSimilarity(train_p, model_p, num_best=5)
print(instancia_p)

WmdSimilarity<867 docs, 100 features>
